In [ ]:
"""
Created on Thu Feb 08 10:42 2024

Look at some 2D variables in the Casimir runs

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'

In [5]:
file_open_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
file_closed_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')

In [6]:
file_open_mean10 = file_open_mean10.where(file_open_mean10['so'] > 0)


In [7]:
lon = file_open_mean10.nav_lon
lat = file_open_mean10.nav_lat

In [8]:
T_30W_closed = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')
T_30W_open = file_open_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')

In [16]:
ref = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).isel(deptht=0)
plt.figure()

llon = lon
llat = lat

proj = ccrs.PlateCarree(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [48]:
def compute_density(da):
    depth = da['e3t'].cumsum('deptht')
    S = da['so']
    T = da['thetao']
    return gsw.rho(S,T,depth)

In [52]:
def compute_sigma2(da):
    S = da['so']
    T = da['thetao']
    return gsw.density.sigma2(S, T)

In [59]:
def transect_along_ocean(var, lon, lon_min, lon_max, cpalette, varname):

    plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max)).mean('x')
    plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max)).mean('x')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*3))

    abs1 = ax[0].contourf(plot_var_closed.y,-1*plot_var_closed.deptht, plot_var_closed, cmap=cpalette)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title(varname)
    
    abs2 = ax[1].contourf(plot_var_open.y,-1*plot_var_open.deptht, plot_var_open, cmap=cpalette)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].contourf(plot_var_open.y,-1*plot_var_open.deptht, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

In [ ]:
### ATLANTIC

In [60]:
f_temp = transect_along_ocean('thetao', lon, -31., -29.,cmocean.cm.thermal,'Conservative Temperature [°C]')

In [55]:
f_sal = transect_along_ocean('so', lon, -31., -29.,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [56]:
file_closed_mean10['rho'] = compute_density(file_closed_mean10).load()
file_open_mean10['rho'] = compute_density(file_open_mean10).load()

In [57]:
f_rho = transect_along_ocean('rho', lon, -31., -29.,cmocean.cm.dense,'In-situ density [kg/m3]')

In [58]:
file_closed_mean10['sigma2'] = compute_sigma2(file_closed_mean10).load()
file_open_mean10['sigma2'] = compute_sigma2(file_open_mean10).load()

In [62]:
f_sigma2 = transect_along_ocean('sigma2', lon, -31., -29.,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')

In [ ]:
### PACIFIC

In [63]:
f_temp = transect_along_ocean('thetao', lon, -171, -169.,cmocean.cm.thermal,'Conservative Temperature [°C]')

In [64]:
f_sal = transect_along_ocean('so', lon,  -171, -169.,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [65]:
f_rho = transect_along_ocean('rho', lon, -171, -169.,cmocean.cm.dense,'In-situ density [kg/m3]')

In [66]:
f_sigma2 = transect_along_ocean('sigma2', lon, -171, -169.,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')